In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

文本情感分类

In [2]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes) # 3*3 = 9
        self.W = nn.Embedding(vocab_size, embedding_size) # 16,2 
        self.Weight = nn.Linear(self.num_filters_total, num_classes, bias=False) # 9,2
        self.Bias = nn.Parameter(torch.ones([num_classes])) # 2
        self.filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (size, embedding_size)) for size in filter_sizes]) # 1,3,2,2
        
    def forward(self, X):
        # X 即 inputs (6,3)
        embedded_chars = self.W(X)   # [batch_size, sequence_length, sequence_length]
        # embedded_chars (6,3,2)
        embedded_chars = embedded_chars.unsqueeze(1)
        # embedded_chars (6,1,3,2) 
        # batch_size = 6, channels = 1, height_1 = 3, width_1 = 2
        
        pooled_outputs = []
        for i, conv in enumerate(self.filter_list):
            # conv (1,3 kernel_size=(2,2), stride=(1,1))
            # in_channels = 1, out_channels = 3, height_2 = 2, width_2 = 2, stride步长为默认值 1
            h = F.relu(conv(embedded_chars))
            # h (6,3,2,1) 6为batchsize，3为out_channels，2,1是 (3,2)被kernnel(2,2)采样之后的结果
            mp = nn.MaxPool2d((sequence_length - filter_sizes[i] + 1, 1)) # (2,1)     kernel为(2,1)
            pooled = mp(h).permute(0, 3, 2, 1)  # (6,3,1,1) -> (6,1,1,3)
            # permute 将维度换位
            pooled_outputs.append(pooled)
        
        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # (6,1,1,9)
        # torch.cat 是将list类型的pooled_outputs,按照index 3进行 tensor的拼接，所以 3个(6,1,1,3)按照index3拼接后为 (6,1,1,9)
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])  # (6,9)
        model = self.Weight(h_pool_flat) + self.Bias  # (6,2)
        # 输出是(6,2) 6个batchsize,2个类别（情感极性）
        return model        

In [3]:
embedding_size = 2
sequence_length = 3 
num_classes = 2 # number of classes
filter_sizes = [2, 2, 2] # n-gram windows
num_filters = 3 # number of filters
# 3 words sentences
sentences = ['i love you', 'he loves me', 'she likes baseball', 'i hate you', 'sorry for that', 'this is awful']
labels = [1, 1, 1, 0, 0, 0] # 1 is good, 0 is not good

In [15]:
word_list = ' '.join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [16]:
model = TextCNN()

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# inputs (6,3)
inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
targets = torch.LongTensor([out for out in labels])
# targets (6)

In [19]:
for epoch in range(5000):
    optimizer.zero_grad()
    # output (6,2)
    output = model(inputs)  # [batch_size, num_classes]
    
    loss = criterion(output, targets)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch+1), 'cost=', '{:.6f}'.format(loss))
    
    loss.backward()
    optimizer.step()    

Epoch: 1000 cost= 0.002719
Epoch: 2000 cost= 0.000470
Epoch: 3000 cost= 0.000160
Epoch: 4000 cost= 0.000069
Epoch: 5000 cost= 0.000033


In [20]:
test_text = 'sorry hate she'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)

In [21]:
predict = model(test_batch).data.max(1, keepdim=True)[1]

In [22]:
if predict[0][0] == 0:
    print(test_text, 'is bad mean.')
else:
    print(test_text, 'is godd mean.')

sorry hate she is bad mean.
